<a href="https://colab.research.google.com/github/isaac-ron/crisisconnectmodels/blob/main/crisisconnect_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import evaluate

# --- 1. Load your COMBINED 1637-entry dataset ---
try:
    df = pd.read_csv('crisis_tweets.csv')
except FileNotFoundError:
    print("Error: 'crisis_tweets.csv' not found.")
    print("Please ensure your combined file is uploaded and named correctly.")
    raise

print(f"Total entries loaded: {len(df)}")

# --- FIX 1: Remove the "dirty" row where the column contains its own header ---
# This assumes the header row is the only row where 'crisis_detection' is 'crisis_detection'
df = df[df['crisis_detection'] != 'crisis_detection'].copy() # Added .copy()
print(f"Entries after removing header row: {len(df)}")

# --- FIX 2: Map the *strings* "True" and "False" to 1 and 0, also handle booleans ---
# This will create NaNs for any other junk data
label_map = {
    'True': 1,
    'False': 0,
    True: 1,  # Also handle if some are already boolean
    False: 0  # Also handle if some are already boolean
}
df['label'] = df['crisis_detection'].map(label_map)

# --- FIX 3: Now, drop any rows that failed the mapping (are NaN) ---
df.dropna(subset=['label'], inplace=True)
print(f"Entries after dropping all bad labels: {len(df)}")

# --- Convert the clean 'label' column to int ---
df['label'] = df['label'].astype(int)

# --- 3. Define BINARY label mappings ---
binary_label2id = {"Non-Crisis": 0, "Crisis": 1}
binary_id2label = {0: "Non-Crisis", 1: "Crisis"}
labels_list = ["Non-Crisis", "Crisis"]

# --- 4. Split the data ---
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label'] # This will now work
)

print(f"Training samples: {len(train_df)}")
print(f"Testing samples: {len(test_df)}")

# --- 5. Convert to Hugging Face Dataset object ---
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
ds = DatasetDict({'train': train_dataset, 'test': test_dataset})

print("Dataset created:")
print(ds)

# --- 6. Load Tokenizer ---
model_name = "crisistransformers/CT-M1-Complete"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# --- 7. Tokenize Function ---
def tokenize_function(examples):
    # Add a check for None or NaN in tweet_text before tokenizing
    texts = [text if isinstance(text, str) and pd.notna(text) else "" for text in examples['tweet_text']]
    return tokenizer(texts, padding="max_length", truncation=True)


tokenized_ds = ds.map(tokenize_function, batched=True)

# --- 8. Calculate BINARY Class Weights ---
train_labels = np.array(train_df['label'])
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

if torch.cuda.is_available():
    class_weights_tensor = class_weights_tensor.to('cuda')
print(f"Binary Class Weights: {class_weights_tensor}")

# --- 9. Define the WeightedTrainer Class ---
class WeightedTrainer(Trainer):
    # Added num_items_in_batch to the signature to fix the TypeError
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=0):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# --- 10. Load the BINARY Model ---
model_binary = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels_list), # 2 labels
    id2label=binary_id2label,
    label2id=binary_label2id
)

# --- 11. Define Training Arguments ---
training_args_binary = TrainingArguments(
    output_dir="binary_crisis_classifier",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# --- 12. Define Compute Metrics ---
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    # Use 'macro' for multi-class classification (severity) or 'binary' for binary (crisis detection)
    # Since this function is used by both trainers, 'macro' is a safer default
    # or you might need separate compute_metrics functions. Let's use 'macro' for now.
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro") # Changed to 'macro'
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}

# --- 13. Initialize the Trainer ---
# Corrected FutureWarning by using processing_class
trainer_binary = WeightedTrainer(
    model=model_binary,
    args=training_args_binary,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer, # Still keep tokenizer here for now as processing_class is new
    compute_metrics=compute_metrics,
)

# --- 14. Train! ---
print("Starting training for the BINARY 'Gatekeeper' model...")
trainer_binary.train()

# --- 15. Save the Final Model ---
trainer_binary.save_model("my_final_binary_model")
print("Binary 'Gatekeeper' model saved to 'my_final_binary_model'")

Total entries loaded: 1637
Entries after removing header row: 1636
Entries after dropping all bad labels: 1636
Training samples: 1308
Testing samples: 328
Dataset created:
DatasetDict({
    train: Dataset({
        features: ['tweet_text', 'crisis_detection', 'severity_score', 'crisis_type', 'location', 'label', '__index_level_0__'],
        num_rows: 1308
    })
    test: Dataset({
        features: ['tweet_text', 'crisis_detection', 'severity_score', 'crisis_type', 'location', 'label', '__index_level_0__'],
        num_rows: 328
    })
})


Map:   0%|          | 0/1308 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/328 [00:00<?, ? examples/s]

Binary Class Weights: tensor([0.7947, 1.3485], device='cuda:0')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at crisistransformers/CT-M1-Complete and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3787897733.py:145: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer_binary = WeightedTrainer(


Starting training for the BINARY 'Gatekeeper' model...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.107934,0.978659,0.977043
2,No log,0.097383,0.972561,0.970871
3,No log,0.080859,0.981707,0.980357
4,No log,0.057759,0.987805,0.986992
5,No log,0.056821,0.987805,0.986992


Binary 'Gatekeeper' model saved to 'my_final_binary_model'


In [ ]:
# 'df' should still be your full, clean 1637-entry DataFrame
# 1. Filter for crisis tweets ONLY
df_severity = df[df['label'] == 1].copy() # label 1 = 'Crisis'

print(f"Total crisis-only tweets for severity training: {len(df_severity)}")

# 2. Create the new severity labels (Low, Medium, High, Critical)
# We can just use the 'severity_score' column directly
sev_labels_list = sorted(df_severity['severity_score'].unique().tolist())
sev_label2id = {label: i for i, label in enumerate(sev_labels_list)}
sev_id2label = {i: label for i, label in enumerate(sev_labels_list)}

df_severity['label'] = df_severity['severity_score'].map(sev_label2id)

print(f"Severity labels: {sev_label2id}")

# 3. Split this new dataset
sev_train_df, sev_test_df = train_test_split(
    df_severity,
    test_size=0.2, # 20% of your crisis-only data
    random_state=42,
    stratify=df_severity['label']
)

# 4. Convert to Hugging Face Dataset
sev_train_dataset = Dataset.from_pandas(sev_train_df)
sev_test_dataset = Dataset.from_pandas(sev_test_df)
ds_sev = DatasetDict({'train': sev_train_dataset, 'test': sev_test_dataset})

print(ds_sev)

Total crisis-only tweets for severity training: 607
Severity labels: {'Critical': 0, 'High': 1, 'Low': 2, 'Medium': 3}
DatasetDict({
    train: Dataset({
        features: ['tweet_text', 'crisis_detection', 'severity_score', 'crisis_type', 'location', 'label', '__index_level_0__'],
        num_rows: 485
    })
    test: Dataset({
        features: ['tweet_text', 'crisis_detection', 'severity_score', 'crisis_type', 'location', 'label', '__index_level_0__'],
        num_rows: 122
    })
})


In [ ]:
# 1. Tokenize the severity dataset
# (This re-uses your 'tokenize_function')
tokenized_ds_sev = ds_sev.map(tokenize_function, batched=True)

# 2. Calculate NEW class weights for the 4 severity levels
sev_train_labels = np.array(sev_train_df['label'])
class_weights_sev = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(sev_train_labels),
    y=sev_train_labels
)

# *** IMPORTANT ***
# We must assign these new weights to the global 'class_weights_tensor'
# variable so that our 'WeightedTrainer' class will use them.
class_weights_tensor = torch.tensor(class_weights_sev, dtype=torch.float).to('cuda')

print(f"Severity Class Weights: {class_weights_tensor}")

Map:   0%|          | 0/485 [00:00<?, ? examples/s]

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

Severity Class Weights: tensor([1.0636, 0.7723, 2.7557, 0.7132], device='cuda:0')


In [ ]:
# 1. Load a FRESH model for SEVERITY classification (4 labels)
model_severity = AutoModelForSequenceClassification.from_pretrained(
    model_name, # "crisistransformers/CT-M1-Complete"
    num_labels=len(sev_labels_list), # Should be 4
    id2label=sev_id2label,
    label2id=sev_label2id
)

# --- Recalculate Severity Class Weights ---
# Ensure sev_train_df is available from previous steps
# Assuming sev_train_df['label'] contains the integer labels for the 4 severity classes
# If not, you might need to create a 'label' column in sev_train_df mapping the severity strings to integers (0-3)
# based on sev_label2id *before* this point.
# Based on cell iVYWOiNPhcrw, sev_train_df already has an integer 'label' column.

sev_train_labels = np.array(sev_train_df['label'])
class_weights_sev = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(sev_train_labels),
    y=sev_train_labels
)

# *** IMPORTANT ***
# Re-assign these new severity weights to the global 'class_weights_tensor'
# variable so that our 'WeightedTrainer' class will use them for this training run.
class_weights_tensor = torch.tensor(class_weights_sev, dtype=torch.float).to('cuda')

print(f"Severity Class Weights (recalculated in this cell): {class_weights_tensor}")
# --- End Recalculation ---


# 2. Define new Training Arguments
training_args_sev = TrainingArguments(
    output_dir="severity_classifier",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8, # This smaller dataset might need more epochs
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# 3. Initialize the WeightedTrainer
# (It will automatically use the new 'class_weights_tensor' we just set)
trainer_sev = WeightedTrainer(
    model=model_severity,
    args=training_args_sev,
    train_dataset=tokenized_ds_sev["train"],
    eval_dataset=tokenized_ds_sev["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, # Re-uses your metric function
)

# 4. Train!
print("Starting training for the SEVERITY 'Expert' model...")
trainer_sev.train()

# 5. Save the final model
trainer_sev.save_model("my_final_severity_model")
print("Severity 'Expert' model saved to 'my_final_severity_model'")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at crisistransformers/CT-M1-Complete and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1379650802.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer_sev = WeightedTrainer(


Severity Class Weights (recalculated in this cell): tensor([1.0636, 0.7723, 2.7557, 0.7132], device='cuda:0')
Starting training for the SEVERITY 'Expert' model...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.281088,0.590164,0.571894
2,No log,1.145226,0.663934,0.649013
3,No log,1.018132,0.680328,0.669159
4,No log,0.919420,0.680328,0.678179
5,No log,0.859232,0.680328,0.682642
6,No log,0.829702,0.680328,0.682362
7,No log,0.818873,0.696721,0.699459
8,No log,0.803468,0.704918,0.701993


Severity 'Expert' model saved to 'my_final_severity_model'


NEW SEVERITY MODEL dont use the one above


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import evaluate

# --- 1. Load ALL your data ---
try:
    df_main = pd.read_csv('crisis_tweets.csv')
    df_booster_sev = pd.read_csv('booster_severity.csv')
except FileNotFoundError:
    print("Error: Make sure 'crisis_tweets.csv' and 'booster_severity.csv' are both uploaded.")
    raise

# Combine them
df_combined = pd.concat([df_main, df_booster_sev], ignore_index=True)
print(f"Total combined entries: {len(df_combined)}")

# --- 2. Clean and Filter for REAL Crises ---
# Clean the 'crisis_detection' column
# Explicitly map boolean values and handle the 'crisis_detection' string
def map_crisis_detection_to_int(value):
    if value is True:
        return 1
    elif value is False:
        return 0
    else:
        return np.nan # Treat anything else (like the header string) as NaN

df_combined['is_crisis_label'] = df_combined['crisis_detection'].apply(map_crisis_detection_to_int)


# Now, filter for REAL crises only (is_crisis_label == 1) and drop NaNs in severity_score
df_severity = df_combined[df_combined['is_crisis_label'] == 1].copy()
df_severity = df_severity.dropna(subset=['severity_score']).copy() # Use .copy()
df_severity = df_severity[df_severity['severity_score'].isin(['Low', 'Medium', 'High', 'Critical'])].copy() # Use .copy()


print(f"Total REAL crisis tweets for severity training: {len(df_severity)}")

# --- 3. Prepare Severity Labels ---
sev_labels_list = sorted(df_severity['severity_score'].unique().tolist())
sev_label2id = {label: i for i, label in enumerate(sev_labels_list)}
sev_id2label = {i: label for i, label in enumerate(sev_labels_list)}
df_severity['label'] = df_severity['severity_score'].map(sev_label2id)

print(f"Severity labels: {sev_label2id}")

# --- 4. Split and Create Dataset ---
sev_train_df, sev_test_df = train_test_split(
    df_severity,
    test_size=0.2,
    random_state=42,
    stratify=df_severity['label']
)

# *** FIX *** Drop the original 'crisis_detection' and 'is_crisis_label' columns before converting to Dataset
# These columns have mixed types and are not needed for the severity model training
sev_train_df = sev_train_df.drop(columns=['crisis_detection', 'is_crisis_label'])
sev_test_df = sev_test_df.drop(columns=['crisis_detection', 'is_crisis_label'])


ds_sev = DatasetDict({
    'train': Dataset.from_pandas(sev_train_df),
    'test': Dataset.from_pandas(sev_test_df)
})
print(ds_sev)

# --- 5. Tokenize and Get Weights ---
# (This assumes 'tokenizer' and 'tokenize_function' are in memory)
tokenized_ds_sev = ds_sev.map(tokenize_function, batched=True)

sev_train_labels = np.array(sev_train_df['label'])
class_weights_sev = compute_class_weight(
    'balanced',
    classes=np.unique(sev_train_labels),
    y=sev_train_labels
)
# *** This is the important part ***
# We assign the new severity weights to the global tensor
class_weights_tensor = torch.tensor(class_weights_sev, dtype=torch.float).to('cuda')
print(f"NEW Severity Class Weights: {class_weights_tensor}")

# --- 6. Load a FRESH Model for Severity ---
# (This assumes 'model_name' is still "crisistransformers/CT-M1-Complete")
model_severity = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(sev_labels_list), # 4 labels
    id2label=sev_id2label,
    label2id=sev_label2id
)

# --- 7. Define Training Arguments ---
training_args_sev = TrainingArguments(
    output_dir="severity_classifier_v2", # New directory
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8, # Train for 8 epochs on this new, larger dataset
    weight_decay=0.01,
    eval_strategy="epoch", # Corrected argument name
    save_strategy="epoch", # Corrected argument name
    load_best_model_at_end=True,
)

# --- 8. Initialize and Train ---
# (This assumes 'WeightedTrainer' class and 'compute_metrics' function are in memory)
trainer_sev = WeightedTrainer(
    model=model_severity,
    args=training_args_sev,
    train_dataset=tokenized_ds_sev["train"],
    eval_dataset=tokenized_ds_sev["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, # Re-uses your metric function
)

print("Starting training for the NEW 'Severity Expert' model (v2)...")
trainer_sev.train()

# --- 9. Save the Final Model ---
trainer_sev.save_model("my_final_severity_model_v2")
print("New 'Severity Expert' model (v2) saved.")

Total combined entries: 1820
Total REAL crisis tweets for severity training: 179
Severity labels: {'Critical': 0, 'High': 1, 'Low': 2, 'Medium': 3}
DatasetDict({
    train: Dataset({
        features: ['tweet_text', 'severity_score', 'crisis_type', 'location', 'label', '__index_level_0__'],
        num_rows: 143
    })
    test: Dataset({
        features: ['tweet_text', 'severity_score', 'crisis_type', 'location', 'label', '__index_level_0__'],
        num_rows: 36
    })
})


Map:   0%|          | 0/143 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

NEW Severity Class Weights: tensor([1.1172, 1.0833, 0.8938, 0.9408], device='cuda:0')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at crisistransformers/CT-M1-Complete and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1673074893.py:116: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer_sev = WeightedTrainer(


Starting training for the NEW 'Severity Expert' model (v2)...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.378304,0.250000,0.172697
2,No log,1.372766,0.277778,0.217409
3,No log,1.366038,0.333333,0.262446
4,No log,1.357335,0.305556,0.215839
5,No log,1.348230,0.333333,0.292991
6,No log,1.339035,0.361111,0.333188
7,No log,1.329486,0.333333,0.310483
8,No log,1.327030,0.333333,0.310483


New 'Severity Expert' model (v2) saved.


FINAL SEVERITY MODEL


In [ ]:
# Zip the binary model directory
!zip -r my_final_binary_model.zip my_final_binary_model/

# Zip the 4-class severity model directory
!zip -r my_final_severity_model_4_class.zip my_final_severity_model_4_class/

print("Model directories zipped. You can now download 'my_final_binary_model.zip' and 'my_final_severity_model_4_class.zip' from the files sidebar.")

  adding: my_final_binary_model/ (stored 0%)
  adding: my_final_binary_model/config.json (deflated 52%)
  adding: my_final_binary_model/training_args.bin (deflated 53%)
  adding: my_final_binary_model/tokenizer.json (deflated 81%)
  adding: my_final_binary_model/merges.txt (deflated 51%)
  adding: my_final_binary_model/tokenizer_config.json (deflated 75%)
  adding: my_final_binary_model/special_tokens_map.json (deflated 84%)
  adding: my_final_binary_model/vocab.json (deflated 57%)
  adding: my_final_binary_model/model.safetensors (deflated 7%)
  adding: my_final_severity_model_4_class/ (stored 0%)
  adding: my_final_severity_model_4_class/config.json (deflated 52%)
  adding: my_final_severity_model_4_class/training_args.bin (deflated 53%)
  adding: my_final_severity_model_4_class/tokenizer.json (deflated 81%)
  adding: my_final_severity_model_4_class/merges.txt (deflated 51%)
  adding: my_final_severity_model_4_class/tokenizer_config.json (deflated 75%)
  adding: my_final_severity_mod

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import evaluate

# --- 1. Load your combined 1637-entry dataset ---
# (This assumes 'df_combined' is still in memory from the last run)
# If not, re-load your crisis_tweets.csv and booster_severity.csv here.
print(f"Total combined entries: {len(df_combined)}")

# --- 2. Clean and Filter for REAL Crises ---
# (This is the same cleaning logic as before)
label_map = {
    'True': 1, 'False': 0,
    True: 1, False: 0,
    'crisis_detection': np.nan
}
df_combined['is_crisis_label'] = df_combined['crisis_detection'].map(label_map)
df_combined = df_combined.dropna(subset=['is_crisis_label'])
df_severity = df_combined[df_combined['is_crisis_label'] == 1].copy()
df_severity = df_severity.dropna(subset=['severity_score'])
df_severity = df_severity[df_severity['severity_score'].isin(['Low', 'Medium', 'High', 'Critical'])]

print(f"Total REAL crisis tweets: {len(df_severity)}")

# --- 3. Prepare 4-Class Labels ---
# (This is the same 4-class setup as the last failed run)
sev_labels_list = sorted(df_severity['severity_score'].unique().tolist())
sev_label2id = {label: i for i, label in enumerate(sev_labels_list)}
sev_id2label = {i: label for i, label in enumerate(sev_labels_list)}
df_severity['label'] = df_severity['severity_score'].map(sev_label2id)

print(f"Severity labels: {sev_label2id}")

# --- 4. Split and Create 4-Class Dataset ---
sev_train_df, sev_test_df = train_test_split(
    df_severity,
    test_size=0.2,
    random_state=42,
    stratify=df_severity['label']
)

ds_sev_4_class = DatasetDict({
    'train': Dataset.from_pandas(sev_train_df[['tweet_text', 'label']]),
    'test': Dataset.from_pandas(sev_test_df[['tweet_text', 'label']])
})
print(ds_sev_4_class)

# --- 5. Tokenize and Get 4-Class Weights ---
tokenized_ds_sev_4_class = ds_sev_4_class.map(tokenize_function, batched=True)

sev_train_labels = np.array(sev_train_df['label'])
class_weights_sev_4 = compute_class_weight(
    'balanced',
    classes=np.unique(sev_train_labels),
    y=sev_train_labels
)
# *** Update the global tensor with these 4-class weights ***
class_weights_tensor = torch.tensor(class_weights_sev_4, dtype=torch.float).to('cuda')
print(f"4-Class Severity Class Weights: {class_weights_tensor}")

# --- 6. Load a FRESH Model for 4-Class Classification ---
model_severity_4_class = AutoModelForSequenceClassification.from_pretrained(
    model_name, # "crisistransformers/CT-M1-Complete"
    num_labels=len(sev_labels_list), # Should be 4
    id2label=sev_id2label,
    label2id=sev_label2id
)

# --- 7. Define Training Arguments (with MORE EPOCHS) ---
training_args_sev_4 = TrainingArguments(
    output_dir="severity_classifier_4_class", # New directory
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,  # <<--- TRAINING FOR 15 EPOCHS
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# --- 8. Initialize and Train ---
# (This re-uses your 'WeightedTrainer' and 'compute_metrics' functions)
trainer_sev_4 = WeightedTrainer(
    model=model_severity_4_class,
    args=training_args_sev_4,
    train_dataset=tokenized_ds_sev_4_class["train"],
    eval_dataset=tokenized_ds_sev_4_class["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Starting training for the 4-Class 'Severity Expert' (15 epochs)...")
trainer_sev_4.train()

# --- 9. Save the Final Model ---
trainer_sev_4.save_model("my_final_severity_model_4_class")
print("New 4-Class 'Severity Expert' model saved.")

Total combined entries: 1815
Total REAL crisis tweets: 786
Severity labels: {'Critical': 0, 'High': 1, 'Low': 2, 'Medium': 3}
DatasetDict({
    train: Dataset({
        features: ['tweet_text', 'label', '__index_level_0__'],
        num_rows: 628
    })
    test: Dataset({
        features: ['tweet_text', 'label', '__index_level_0__'],
        num_rows: 158
    })
})


/tmp/ipython-input-3186642634.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['is_crisis_label'] = df_combined['crisis_detection'].map(label_map)


Map:   0%|          | 0/628 [00:00<?, ? examples/s]

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

4-Class Severity Class Weights: tensor([1.0753, 0.8307, 1.8690, 0.7512], device='cuda:0')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at crisistransformers/CT-M1-Complete and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3186642634.py:94: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer_sev_4 = WeightedTrainer(


Starting training for the 4-Class 'Severity Expert' (15 epochs)...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.252771,0.518987,0.521522
2,No log,1.051672,0.664557,0.691886
3,No log,0.848189,0.670886,0.674267
4,No log,0.721425,0.715190,0.727750
5,No log,0.626280,0.778481,0.786771
6,No log,0.609760,0.765823,0.771847
7,No log,0.621723,0.765823,0.774699
8,No log,0.586703,0.759494,0.765156
9,No log,0.564218,0.765823,0.773042
10,No log,0.552707,0.810127,0.812226


New 4-Class 'Severity Expert' model saved.


In [ ]:
import os
import shutil

# List of directories containing intermediate checkpoints to delete
# Added 'severity_classifier_4_class' from the last training run
checkpoint_dirs_to_delete = [
    "severity_classifier_4_class"
]

print("Deleting intermediate checkpoint directories...")

for dir_name in checkpoint_dirs_to_delete:
    if os.path.exists(dir_name):
        try:
            shutil.rmtree(dir_name)
            print(f"Deleted: {dir_name}")
        except OSError as e:
            print(f"Error deleting directory {dir_name}: {e}")
    else:
        print(f"Directory not found: {dir_name}")

print("Deletion complete.")

Deleting intermediate checkpoint directories...
Deleted: severity_classifier_4_class
Deletion complete.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import shutil

# List of directories containing intermediate checkpoints
checkpoint_dirs = [
    "binary_crisis_classifier",
    "severity_classifier",
    "severity_classifier_v2"
]

print("Deleting intermediate checkpoint directories...")

for dir_name in checkpoint_dirs:
    if os.path.exists(dir_name):
        try:
            shutil.rmtree(dir_name)
            print(f"Deleted: {dir_name}")
        except OSError as e:
            print(f"Error deleting directory {dir_name}: {e}")
    else:
        print(f"Directory not found: {dir_name}")

print("Deletion complete.")

Deleting intermediate checkpoint directories...
Deleted: binary_crisis_classifier
Deleted: severity_classifier
Deleted: severity_classifier_v2
Deletion complete.


In [ ]:
from transformers import pipeline
import re
import json

# --- 1. Load the "Gatekeeper" (Binary Classifier) 🧠 ---
print("Loading the 'Gatekeeper' (Binary) model...")
gatekeeper_pipeline = pipeline(
    "text-classification",
    model="my_final_binary_model"
)

# --- 2. Load the "Severity Expert" (4-Class Classifier) 🧠 ---
print("Loading the 'Severity Expert' model...")
# --- UPDATED: Use the new 4-class severity model ---
severity_pipeline = pipeline(
    "text-classification",
    model="my_final_severity_model_4_class" # Changed model path
)

# --- 3. Load the "Location Extractor" (NER) 📍 ---
print("Loading the 'Location Extractor' (NER) model...")
ner_pipeline = pipeline(
    "ner",
    model="dslim/bert-base-NER",
    grouped_entities=True
)

print("All models loaded. Pipeline is ready!")

Loading the 'Gatekeeper' (Binary) model...


Device set to use cuda:0


Loading the 'Severity Expert' model...


Device set to use cuda:0


Loading the 'Location Extractor' (NER) model...


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


All models loaded. Pipeline is ready!


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [ ]:
def analyze_crisis_tweet(text):
    """
    Analyzes a tweet using the 2-step pipeline.
    """
    # --- Step 1: Run the "Gatekeeper" model ---
    gatekeeper_result = gatekeeper_pipeline(text)[0]

    # --- Step 2: Check the Gatekeeper's decision ---
    if gatekeeper_result['label'] == 'Non-Crisis':
        # If it's not a crisis, we stop here.
        return {
            "tweet_text": text,
            "is_crisis": False,
            "severity": "N/A",
            "classification_confidence": round(gatekeeper_result['score'], 4),
            "extracted_locations": [] # No need to run NER
        }

    # --- Step 3: If it IS a crisis, run the "Expert" models ---

    # A) Run the Severity Expert
    severity_result = severity_pipeline(text)[0]
    severity = severity_result['label']

    # B) Run the Location Extractor
    ner_results = ner_pipeline(text)
    locations = []
    for entity in ner_results:
        if entity['entity_group'] == 'LOC':
            clean_loc = re.sub(r'[#@]', '', entity['word'])
            locations.append(clean_loc.strip())

    # C) Format the final output
    return {
        "tweet_text": text,
        "is_crisis": True,
        "severity": severity,
        "classification_confidence": round(severity_result['score'], 4),
        "extracted_locations": list(set(locations))
    }

In [ ]:
# --- Let's test it! ---

# A real crisis
test_tweet_1 = "Horrible multi-car pile-up on the A104 highway near Naivasha. Traffic is backed up for kilometers. Avoid the area."

# Figurative language (non-crisis)
test_tweet_2 = "My CPU is on fire trying to render this 4K video project. 💻🔥 #TechProbs"

# A real, but lower-severity, crisis
test_tweet_3 = "Just felt a small shake in Nakuru! The house was rattling for a second. #earthquake"

# Figurative language
test_tweet_4 = "That new Sauti Sol album just dropped and it's an absolute bombshell!"

# A critical event
test_tweet_5 = "Active shooter reported at Two Rivers Mall. Police are on the scene. Stay away from the area!"


print(json.dumps(analyze_crisis_tweet(test_tweet_1), indent=2))
print("---")
print(json.dumps(analyze_crisis_tweet(test_tweet_2), indent=2))
print("---")
print(json.dumps(analyze_crisis_tweet(test_tweet_3), indent=2))
print("---")
print(json.dumps(analyze_crisis_tweet(test_tweet_4), indent=2))
print("---")
print(json.dumps(analyze_crisis_tweet(test_tweet_5), indent=2))

{
  "tweet_text": "Horrible multi-car pile-up on the A104 highway near Naivasha. Traffic is backed up for kilometers. Avoid the area.",
  "is_crisis": true,
  "severity": "High",
  "classification_confidence": 0.8142,
  "extracted_locations": [
    "A1",
    "Naivasha"
  ]
}
---
{
  "tweet_text": "My CPU is on fire trying to render this 4K video project. \ud83d\udcbb\ud83d\udd25 #TechProbs",
  "is_crisis": false,
  "severity": "N/A",
  "classification_confidence": 0.9898,
  "extracted_locations": []
}
---
{
  "tweet_text": "Just felt a small shake in Nakuru! The house was rattling for a second. #earthquake",
  "is_crisis": true,
  "severity": "Medium",
  "classification_confidence": 0.8383,
  "extracted_locations": []
}
---
{
  "tweet_text": "That new Sauti Sol album just dropped and it's an absolute bombshell!",
  "is_crisis": false,
  "severity": "N/A",
  "classification_confidence": 0.9897,
  "extracted_locations": []
}
---
{
  "tweet_text": "Active shooter reported at Two Rivers Ma